# Phase 6 : IA Responsable - Équité, Transparence et Monitoring

## Objectifs (Module 4)
- Analyser l'équité du modèle (fairness)
- Assurer la transparence des décisions (SHAP, Grad-CAM)
- Mettre en place le monitoring (drift detection)
- Calculer le ROI et les KPI business

**⚠️ IMPORTANT** : Ce notebook est OBLIGATOIRE pour le projet

In [ ]:
# Imports
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import confusion_matrix

import sys
sys.path.append('..')
from src.evaluation.metrics import compute_all_metrics
from src.utils.config import load_config

config = load_config('../configs/config.yaml')

## 1. Analyse d'Équité (Fairness)

### Contexte
En diagnostic médical, les biais algorithmiques peuvent causer :
- Soins inéquitables entre groupes
- Erreurs de diagnostic pour certaines populations
- Perte de confiance dans le système

### 1.1 Identification des Biais Potentiels

In [ ]:
# Charger les prédictions et métadonnées
# TODO: Charger predictions_df avec colonnes:
# - patient_id
# - hospital
# - true_pn_stage
# - predicted_pn_stage
# - confidence

predictions_df = pd.read_csv('../results/predictions/test_predictions.csv')

print("Distribution par hôpital:")
print(predictions_df['hospital'].value_counts())

### 1.2 Métriques de Fairness

#### Demographic Parity (Parité Démographique)

In [ ]:
# Proportion de prédictions positives par hôpital
demographic_parity = {}

for hospital in predictions_df['hospital'].unique():
    mask = predictions_df['hospital'] == hospital
    
    # Proportion de patients avec métastases (pN > 0)
    positive_rate = (predictions_df[mask]['predicted_pn_stage'] > 0).mean()
    demographic_parity[hospital] = positive_rate
    
    print(f"Hôpital {hospital}: {positive_rate:.2%} de prédictions positives")

# Visualiser
fig = px.bar(x=list(demographic_parity.keys()), 
             y=list(demographic_parity.values()),
             title='Parité Démographique par Hôpital',
             labels={'x': 'Hôpital', 'y': 'Taux de Prédictions Positives'})
fig.show()

#### Equalized Odds (Égalité des Chances)

In [ ]:
# TPR et FPR par hôpital
equalized_odds = []

for hospital in predictions_df['hospital'].unique():
    mask = predictions_df['hospital'] == hospital
    
    # Binariser : 0 = pN0, 1 = pN1+
    y_true_binary = (predictions_df[mask]['true_pn_stage'] > 0).astype(int)
    y_pred_binary = (predictions_df[mask]['predicted_pn_stage'] > 0).astype(int)
    
    # Matrice de confusion
    cm = confusion_matrix(y_true_binary, y_pred_binary)
    
    # Calculer TPR et FPR
    tpr = cm[1,1] / (cm[1,1] + cm[1,0]) if (cm[1,1] + cm[1,0]) > 0 else 0
    fpr = cm[0,1] / (cm[0,1] + cm[0,0]) if (cm[0,1] + cm[0,0]) > 0 else 0
    
    equalized_odds.append({
        'hospital': hospital,
        'TPR': tpr,
        'FPR': fpr
    })
    
    print(f"Hôpital {hospital}: TPR={tpr:.3f}, FPR={fpr:.3f}")

# Visualiser
df_eo = pd.DataFrame(equalized_odds)
fig = px.scatter(df_eo, x='FPR', y='TPR', text='hospital',
                 title='Equalized Odds par Hôpital',
                 labels={'TPR': 'True Positive Rate', 'FPR': 'False Positive Rate'})
fig.show()

### 1.3 Stratégies de Correction

#### Post-processing : Calibration par Hôpital

In [ ]:
def calibrate_by_hospital(predictions_df):
    """
    Ajuste les seuils de décision pour chaque hôpital
    pour garantir l'équité des performances
    """
    calibrated_predictions = predictions_df.copy()
    
    for hospital in predictions_df['hospital'].unique():
        mask = predictions_df['hospital'] == hospital
        
        # TODO: Optimiser le seuil pour ce hôpital
        # Objectif : maximiser F1 ou minimiser FN
        optimal_threshold = optimize_threshold_for_hospital(predictions_df[mask])
        
        # Appliquer le seuil calibré
        calibrated_predictions.loc[mask, 'calibrated_prediction'] = \
            apply_calibrated_threshold(predictions_df[mask], optimal_threshold)
    
    return calibrated_predictions

# Appliquer la calibration
# calibrated_df = calibrate_by_hospital(predictions_df)
# Comparer les performances avant/après

## 2. Transparence et Explicabilité

### 2.1 SHAP (Niveau Agrégation)

In [ ]:
# Installer si nécessaire : uv sync --group interpretability
import shap

# Charger le modèle d'agrégation (XGBoost)
# TODO: Charger votre modèle XGBoost
# xgb_model = load_model('../models/final/xgboost_aggregator.pkl')

# Charger les features patients
# X_patient_features = pd.read_csv('../data/processed/patient_features.csv')

# Créer l'explainer
# explainer = shap.TreeExplainer(xgb_model)
# shap_values = explainer.shap_values(X_patient_features)

# Visualiser
# shap.summary_plot(shap_values, X_patient_features, plot_type="bar")
# shap.summary_plot(shap_values, X_patient_features)

### 2.2 Grad-CAM (Niveau Patch)

In [ ]:
from src.evaluation.interpretability import GradCAM
from src.visualization.heatmaps import plot_gradcam_heatmap

# Charger le modèle CNN
# model = load_model('../models/final/resnet50_best.pth')

# Sélectionner la couche cible
# target_layer = model.layer4[-1]  # Dernière couche conv

# Initialiser Grad-CAM
# gradcam = GradCAM(model, target_layer)

# Sélectionner des cas représentatifs
# TODO: Charger 10-20 patchs (TP, TN, FP, FN)

# Pour chaque patch
# for patch, label in representative_patches:
#     cam = gradcam.generate_cam(patch, target_class=1)
#     fig = plot_gradcam_heatmap(patch, cam)
#     fig.show()

### 2.3 Feature Importance

In [ ]:
# Feature importance du modèle d'agrégation
# feature_importance = xgb_model.feature_importances_
# feature_names = ['tumor_percentage', 'mean_prob', 'max_prob', 'std_prob', ...]

# Visualiser
# fig = px.bar(x=feature_names, y=feature_importance,
#              title='Importance des Features pour Prédiction pN',
#              labels={'x': 'Feature', 'y': 'Importance'})
# fig.show()

## 3. Monitoring et Drift Detection

### 3.1 Feature Drift

In [ ]:
from scipy.stats import ks_2samp

def detect_feature_drift(X_train, X_prod, threshold=0.05):
    """
    Détecte le drift dans les features entre train et production
    """
    drift_detected = {}
    
    for col in X_train.columns:
        # Test de Kolmogorov-Smirnov
        statistic, p_value = ks_2samp(X_train[col], X_prod[col])
        drift_detected[col] = {
            'p_value': p_value,
            'drift': p_value < threshold
        }
    
    return drift_detected

# Simuler des données de production
# X_train = pd.read_csv('../data/processed/train_features.csv')
# X_prod = pd.read_csv('../data/processed/production_features.csv')

# Détecter le drift
# drift_results = detect_feature_drift(X_train, X_prod)

# Afficher les features avec drift
# for feature, result in drift_results.items():
#     if result['drift']:
#         print(f"⚠️ DRIFT détecté pour {feature}: p-value={result['p_value']:.4f}")

### 3.2 Performance Monitoring

In [ ]:
def monitor_performance_over_time(predictions_history):
    """
    Surveille les performances dans le temps
    """
    performance_metrics = []
    
    for batch in predictions_history:
        metrics = compute_all_metrics(
            batch['y_true'], 
            batch['y_pred']
        )
        metrics['timestamp'] = batch['timestamp']
        performance_metrics.append(metrics)
    
    df_perf = pd.DataFrame(performance_metrics)
    
    # Visualiser l'évolution
    fig = px.line(df_perf, x='timestamp', y=['accuracy', 'recall', 'precision'],
                  title='Évolution des Performances dans le Temps')
    fig.show()
    
    return df_perf

# TODO: Implémenter avec vos données

## 4. ROI et KPI Business

### 4.1 Calcul du ROI

In [ ]:
# Hypothèses (à adapter selon votre contexte)
fn_rate_manual = 0.05  # 5% de faux négatifs en diagnostic manuel
fn_rate_ai = 0.01      # 1% avec votre système IA
cost_per_fn = 50000    # Coût d'un traitement retardé (€)
n_patients_year = 10000  # Nombre de patients par an

# Coûts du système
development_cost = 100000  # Développement (€)
deployment_cost = 20000    # Déploiement (€)
maintenance_cost_year = 10000  # Maintenance annuelle (€)

# Calcul des bénéfices
fn_avoided = (fn_rate_manual - fn_rate_ai) * n_patients_year
cost_avoided_year = fn_avoided * cost_per_fn

# Calcul du ROI sur 3 ans
total_cost = development_cost + deployment_cost + (maintenance_cost_year * 3)
total_benefit = cost_avoided_year * 3

roi = ((total_benefit - total_cost) / total_cost) * 100

print("=== Analyse ROI ===")
print(f"Faux négatifs évités par an : {fn_avoided:.0f} patients")
print(f"Coûts évités par an : {cost_avoided_year:,.0f} €")
print(f"Coût total du système (3 ans) : {total_cost:,.0f} €")
print(f"Bénéfice total (3 ans) : {total_benefit:,.0f} €")
print(f"\n🎯 ROI sur 3 ans : {roi:.1f}%")

### 4.2 KPI Business

In [ ]:
# Définir les KPI
kpi_data = [
    {
        'KPI': 'Taux de détection',
        'Métrique Technique': 'Recall',
        'Valeur': 0.95,
        'Objectif': 0.95,
        'Impact': 'Moins de 5% de métastases manquées'
    },
    {
        'KPI': 'Précision diagnostique',
        'Métrique Technique': 'Precision',
        'Valeur': 0.92,
        'Objectif': 0.90,
        'Impact': 'Moins de 8% de faux positifs'
    },
    {
        'KPI': 'Temps de traitement',
        'Métrique Technique': 'Temps moyen',
        'Valeur': 3.5,  # minutes
        'Objectif': 5.0,
        'Impact': 'Accélération du diagnostic'
    },
]

df_kpi = pd.DataFrame(kpi_data)
print(df_kpi.to_string(index=False))

## 5. Conclusions et Recommandations

### 5.1 Synthèse des Analyses

**Équité** :
- TODO: Résumer les biais identifiés
- TODO: Stratégies de correction appliquées
- TODO: Résultats après correction

**Transparence** :
- TODO: Features les plus importantes identifiées
- TODO: Validation médicale des décisions
- TODO: Cas d'étude documentés

**Monitoring** :
- TODO: Stratégie de drift detection définie
- TODO: Seuils d'alerte établis
- TODO: Plan de réentraînement

**Business** :
- TODO: ROI calculé et justifié
- TODO: KPI définis et mesurés
- TODO: Impact clinique quantifié

### 5.2 Recommandations pour le Déploiement

1. **Workflow Clinique** :
   - Utiliser le système comme outil de pré-screening
   - Validation humaine obligatoire pour cas critiques
   - Double lecture pour cas ambigus

2. **Monitoring Continu** :
   - Vérifier les performances hebdomadairement
   - Détecter le drift mensuellement
   - Réentraîner si dégradation > 5%

3. **Équité** :
   - Auditer les performances par hôpital trimestriellement
   - Recalibrer si disparités détectées
   - Documenter tous les ajustements

4. **Transparence** :
   - Fournir explications Grad-CAM aux pathologistes
   - Documenter les cas d'erreur
   - Maintenir un registre des décisions